In [31]:
import boto3
import io
import csv
from snowflake.snowpark import Session
import datetime

connection_parameters = {
   "account": "dr31778.ca-central-1.aws",
   "user": "PRAVEEN11001",
   "password": "XXXXXXXXXXXXXXXX",
   "role": "ACCOUNTADMIN",  # optional
   "warehouse": "COMPUTE_WH",  # optional
   "database": "DEMO",  # optional
   "schema": "s3"  # optional
}

session = Session.builder.configs(connection_parameters).create()

In [32]:
aws_access_key = 'XXXXXXXXXXXXXXXX'
aws_secret_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
aws_s3_bucket_name = 'praveeng-s3'  
aws_s3_object = 'xmls/'

s3 = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)

In [33]:
def copy_into_sf(session, folder_name, filename):

    copy_date = datetime.datetime.utcnow()
    
    copy_result = session.sql(f"""
    COPY INTO AWS_STUDENT_DATASET_DIR(filename, ID, NAME, NATIONALITY, CITY, 
                              LATITUDE, LONGITUDE, GENDER, ethinic_group, 
                              AGE, ENGLISH_GRADE, MATH_GRADE, SCIENCE_GRADE, 
                              LANGUAGE_GRADE, PORTFOLIO_RATING, COVERLETTER_RATING, 
                              REFLETTER_RATING, copy_date)
    FROM 
        (SELECT '{filename}' AS filename, 
            $1 AS ID, 
            $2 AS NAME, 
            $3 AS NATIONALITY, 
            $4 AS CITY, 
            $5 AS LATITUDE, 
            $6 AS LONGITUDE,
            $7 AS GENDER, 
            $8 AS ethinic_group, 
            $9 AS AGE, 
            $10 AS ENGLISH_GRADE, 
            $11 AS MATH_GRADE, 
            $12 AS SCIENCE_GRADE, 
            $13 AS LANGUAGE_GRADE, 
            $14 AS PORTFOLIO_RATING, 
            $15 AS COVERLETTER_RATING, 
            $16 AS REFLETTER_RATING,
            '{copy_date}' AS copy_date 
        FROM @s3_stage/{folder_name}{filename}.csv)
        
    ON_ERROR='SKIP_FILE'
    FILE_FORMAT= (FORMAT_NAME=CSV_FORMAT);
    """).collect()

    print(copy_result)

In [34]:
# for all formatted csv file 
response = s3.list_objects_v2(Bucket=aws_s3_bucket_name, Prefix=aws_s3_object)

for obj in response['Contents']:
    key = obj['Key']
    if key.endswith('set.csv'):
        filename = key.split('/')[-1].split('.')[0]
        print(filename)
        copy_into_sf(session, aws_s3_object, filename)

student-dataset
[Row(file='s3://praveeng-s3/xmls/student-dataset.csv', status='LOADED', rows_parsed=307, rows_loaded=307, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None)]
